# Imports and Such

In [2]:
from functions.colours          import *
from functions.measurement      import *
from functions.optical_modes    import *
from functions.plotting         import *
from functions.propagation      import *

from LightPipes                 import *

import aotools
import cmath

pmap, imap, customColoursBGY, customColoursViridis = colours()

In [3]:
print("test")

test
